In [ ]:
%load_ext autoreload
%autoreload 2
from common import *

In [ ]:
RESULT_JSON = "/Users/law/repos/viper/results/breakdown/breakdown_revision.json"

In [ ]:
from collections import defaultdict
runs = defaultdict(list)

BMS = get_all_runs(RESULT_JSON)

# pprint(BMS)

In [ ]:
from matplotlib.ticker import (MultipleLocator, AutoMinorLocator)

TIMERS = ("lock", "pmem", "map")

def get_sum_time(bm, op_type):
    total_ns = 0
    for timer in TIMERS:
        total_ns += bm[f"{op_type}-{timer}"]
    return total_ns

st_bm = BMS[0]
at_bm = BMS[1]

st_insert = get_sum_time(BMS[0], "insert")
st_get = get_sum_time(BMS[0], "get")
st_update = get_sum_time(BMS[0], "update")
st_delete = get_sum_time(BMS[0], "delete")

at_insert = get_sum_time(BMS[1], "insert")
at_get = get_sum_time(BMS[1], "get")
at_update = get_sum_time(BMS[1], "update")
at_delete = get_sum_time(BMS[1], "delete")

bar_width = 0.7
st_pos = np.arange(4)
al_pos = [x + bar_width for x in st_pos]
POS = [st_pos, al_pos]

# multi thread op time in us
insert_op_time = 2.35
get_op_time    = 1.11
update_op_time = 2.61
delete_op_time = 2.83


fig, ax = plt.subplots(1, 1, figsize=(2.5, 2.5))

STYLES = {
    'pmem': (PMEM_COLOR, ''),
    'map' : (DRAM_COLOR, ''),
    'lock': ("#990000",  ''),
    'val' : ("#009900",  ''),
}

text_y = 1.05
dur_fs = 18
text_rot = 40

# ALL_RUNS = [(st_bm, st_insert, st_get, st_update, st_delete)] 
ALL_RUNS = [(at_bm, at_insert, at_get, at_update, at_delete)]
for i, (bm, insert, get, update, delete) in enumerate(ALL_RUNS):
    # INSERT
    insert_lock = bm["insert-lock"] / insert
    insert_write = bm["insert-pmem"] / insert
    insert_update = bm["insert-map"] / insert
    ax.bar(POS[i][0], insert_lock, bar_width, bottom=0, 
           label="Fetch/\nLock", color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][0], insert_update, bar_width, bottom=insert_lock, 
           label="Map", color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][0], insert_write, bar_width, bottom=insert_lock + insert_update, 
           label="PMem", color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    ax.text(POS[i][0], text_y, f"{insert_op_time}", ha='center', fontsize=dur_fs, rotation=text_rot)
        
    # GET
    get_map = bm["get-map"] / get
    get_lock = bm["get-lock"] / get
    get_read = bm["get-pmem"] / get
    ax.bar(POS[i][1], get_lock, bar_width, bottom=0, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][1], get_map, bar_width, bottom=get_lock, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][1], get_read, bar_width, bottom=get_map + get_lock,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    ax.text(POS[i][1], text_y, f"{get_op_time}", ha='center', fontsize=dur_fs, rotation=text_rot)
    
    # UPDATE
    update_map = bm["update-map"] / update
    update_lock = bm["update-lock"] / update
    update_modify = bm["update-pmem"] / update
    ax.bar(POS[i][2], update_lock, bar_width, bottom=0, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][2], update_map, bar_width, bottom=update_lock, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][2], update_modify, bar_width, bottom=update_map + update_lock,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    ax.text(POS[i][2], text_y, f"{update_op_time}", ha='center', fontsize=dur_fs, rotation=text_rot)

        
    # DELETE
    delete_lock = bm["delete-lock"] / delete
    delete_write = bm["delete-pmem"] / delete
    delete_update = bm["delete-map"] / delete
    ax.bar(POS[i][3], delete_lock, bar_width, bottom=0, 
           color=STYLES['lock'][0], hatch=STYLES['lock'][1])
    ax.bar(POS[i][3], delete_update, bar_width, bottom=delete_lock, 
           color=STYLES['map'][0], hatch=STYLES['map'][1])
    ax.bar(POS[i][3], delete_write, bar_width, bottom=delete_lock + delete_update,
           color=STYLES['pmem'][0], hatch=STYLES['pmem'][1])
    ax.text(POS[i][3], text_y, f"{delete_op_time}", ha='center', fontsize=dur_fs, rotation=text_rot)
    

ax.set_xticks([r + (0.0 * bar_width) for r in st_pos])
ax.xaxis.set_tick_params(pad=1)
ax.set_xticklabels(["PUT", "GET", "UPDATE", "DELETE"], fontsize=18, rotation=30)

ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
# ax.set_yticklabels(["", 0.2, "", 0.6, "", 1])

ax.set_ylabel("Normalized dur./op", fontsize=20)
ax.yaxis.set_label_coords(-0.28, 0.45)

ax.text(5, 0.9, "←avg. dur.\nin $µs$", ha='center', fontsize=dur_fs + 2)

# Put a legend below current axis
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], loc='center right', bbox_to_anchor=(1.74, 0.4), 
          ncol=1, frameon=False, framealpha=1,
          fontsize=18, columnspacing=0.3, handletextpad=0.2, labelspacing=0.3,
          handlelength=1.8, borderpad=0.1)

for tick in ax.yaxis.get_major_ticks():
    tick.label.set_fontsize(18)

plt.tick_params(axis='x', bottom=False)
ax.set_axisbelow(True)
ax.grid(axis='y', which='major')
hide_border(ax)

fig.savefig('charts/breakdown_half.pdf', bbox_inches='tight')
fig.savefig('charts/breakdown_half.svg', bbox_inches='tight')